# Подключение зависимостей

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ccxt
from IPython.display import clear_output, display
import dill

#sns.set_theme()
%matplotlib inline

In [2]:
import mlflow
from mlflow.tracking import MlflowClient
import mlflavors
import mlforecast.flavor

from statsforecast import StatsForecast
from statsforecast.models import ARCH, GARCH
from mlforecast import MLForecast
from mlforecast.utils import PredictionIntervals
from datasetsforecast import losses
from utilsforecast.plotting import plot_series

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import lightgbm as lgb

# Подключение к MLFlow

In [3]:
mlflow.set_tracking_uri("http://mlflow:5000")

In [31]:
mlflow.set_experiment("btc-usdt_volatility_research")

2024/09/29 11:31:58 INFO mlflow.tracking.fluent: Experiment with name 'btc-usdt_volatility_research' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1727609518788, experiment_id='3', last_update_time=1727609518788, lifecycle_stage='active', name='btc-usdt_volatility_research', tags={}>

# Проведение экспериментов 

## Подготовка данных

In [5]:
# Биржа из которой будут браться данные с помощью CCXT
EXCHANGE = ccxt.okx()
# Инструмент в формате символа для обработки
SYMBOL = "BTC/USDT"
# Таймфрейм свеч
TIMEFRAME = "1h"

In [6]:
from_ts = EXCHANGE.parse8601('2024-01-10 00:00:00')

ohlcv_list = []
ohlcv = EXCHANGE.fetch_ohlcv(symbol=SYMBOL, timeframe=TIMEFRAME, since=from_ts, limit=100)
ohlcv_list.append(ohlcv)

while True:
    from_ts = ohlcv[-1][0]
    new_ohlcv = EXCHANGE.fetch_ohlcv(symbol=SYMBOL, timeframe=TIMEFRAME, since=from_ts, limit=100)
    ohlcv.extend(new_ohlcv)

    print(f"\r{EXCHANGE.iso8601(from_ts)}", end="")
    
    if len(new_ohlcv) <= 1:
    	break

2024-09-29T11:00:00.000Z

In [7]:
ohlcv = pd.DataFrame(ohlcv, columns=["date", "open", "high", "low", "close", "volume"])
ohlcv["date"] = ohlcv["date"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))
ohlcv["close_pct_change"] = ohlcv["close"].pct_change()
ohlcv['close_pct_change'] = ohlcv['close_pct_change'].fillna(0)
ohlcv.reset_index(drop=True)

,date,open,high,low,close,volume,close_pct_change
0,2024-01-10 00:00:00,46109.4,46235.5,45768.3,45854.1,1198.592272,0.000000
1,2024-01-10 01:00:00,45853.9,45955.6,45617.5,45925.8,1463.003867,0.001564
2,2024-01-10 02:00:00,45925.8,46083.0,45878.2,45970.8,640.245350,0.000980
3,2024-01-10 03:00:00,45978.0,46209.9,45923.3,46119.9,359.281616,0.003243
4,2024-01-10 04:00:00,46119.9,46180.9,45941.2,45961.9,380.550457,-0.003426
...,...,...,...,...,...,...,...
6383,2024-09-29 08:00:00,65474.0,65598.9,65440.0,65561.0,121.840340,0.001329
6384,2024-09-29 09:00:00,65560.9,65690.0,65556.7,65582.6,85.416750,0.000329
6385,2024-09-29 10:00:00,65582.0,65588.0,65460.0,65575.6,78.203406,-0.000107
6386,2024-09-29 11:00:00,65575.7,65668.0,65566.2,65653.7,69.346248,0.001191


In [8]:
ohlcv

,date,open,high,low,close,volume,close_pct_change
0,2024-01-10 00:00:00,46109.4,46235.5,45768.3,45854.1,1198.592272,0.000000
1,2024-01-10 01:00:00,45853.9,45955.6,45617.5,45925.8,1463.003867,0.001564
2,2024-01-10 02:00:00,45925.8,46083.0,45878.2,45970.8,640.245350,0.000980
3,2024-01-10 03:00:00,45978.0,46209.9,45923.3,46119.9,359.281616,0.003243
4,2024-01-10 04:00:00,46119.9,46180.9,45941.2,45961.9,380.550457,-0.003426
...,...,...,...,...,...,...,...
6383,2024-09-29 08:00:00,65474.0,65598.9,65440.0,65561.0,121.840340,0.001329
6384,2024-09-29 09:00:00,65560.9,65690.0,65556.7,65582.6,85.416750,0.000329
6385,2024-09-29 10:00:00,65582.0,65588.0,65460.0,65575.6,78.203406,-0.000107
6386,2024-09-29 11:00:00,65575.7,65668.0,65566.2,65653.7,69.346248,0.001191


In [9]:
ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6388 entries, 0 to 6387
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              6388 non-null   datetime64[ns]
 1   open              6388 non-null   float64       
 2   high              6388 non-null   float64       
 3   low               6388 non-null   float64       
 4   close             6388 non-null   float64       
 5   volume            6388 non-null   float64       
 6   close_pct_change  6388 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 349.5 KB


In [10]:
ohlcv.shape

(6388, 7)

In [11]:
ohlcv.nunique()

date                6324
open                6176
high                6066
low                 6040
close               6167
volume              6325
close_pct_change    6322
dtype: int64

In [12]:
ohlcv["date"].astype(int)

0       1704844800000000000
1       1704848400000000000
2       1704852000000000000
3       1704855600000000000
4       1704859200000000000
               ...         
6383    1727596800000000000
6384    1727600400000000000
6385    1727604000000000000
6386    1727607600000000000
6387    1727607600000000000
Name: date, Length: 6388, dtype: int64

В данных могут появится дубликаты, от них надо избавлятся

In [13]:
ohlcv = ohlcv.drop_duplicates(subset=['date'], keep='last')

In [14]:
ohlcv

,date,open,high,low,close,volume,close_pct_change
0,2024-01-10 00:00:00,46109.4,46235.5,45768.3,45854.1,1198.592272,0.000000
1,2024-01-10 01:00:00,45853.9,45955.6,45617.5,45925.8,1463.003867,0.001564
2,2024-01-10 02:00:00,45925.8,46083.0,45878.2,45970.8,640.245350,0.000980
3,2024-01-10 03:00:00,45978.0,46209.9,45923.3,46119.9,359.281616,0.003243
4,2024-01-10 04:00:00,46119.9,46180.9,45941.2,45961.9,380.550457,-0.003426
...,...,...,...,...,...,...,...
6382,2024-09-29 07:00:00,65506.0,65569.9,65440.0,65474.0,176.122269,-0.000487
6383,2024-09-29 08:00:00,65474.0,65598.9,65440.0,65561.0,121.840340,0.001329
6384,2024-09-29 09:00:00,65560.9,65690.0,65556.7,65582.6,85.416750,0.000329
6385,2024-09-29 10:00:00,65582.0,65588.0,65460.0,65575.6,78.203406,-0.000107


In [15]:
train_df = pd.DataFrame(
    columns = ["ds", "y", "unique_id"]   
)

train_df["ds"] = ohlcv["date"].iloc[-324:-24]
train_df["y"] = ohlcv["close_pct_change"].iloc[-324:-24]
train_df["unique_id"] = 1
train_df = train_df.reset_index(drop=True)

In [16]:
train_df

,ds,y,unique_id
0,2024-09-16 00:00:00,-0.007734,1
1,2024-09-16 01:00:00,-0.001096,1
2,2024-09-16 02:00:00,-0.005822,1
3,2024-09-16 03:00:00,0.005324,1
4,2024-09-16 04:00:00,-0.003647,1
...,...,...,...
295,2024-09-28 07:00:00,0.000252,1
296,2024-09-28 08:00:00,-0.002685,1
297,2024-09-28 09:00:00,-0.000154,1
298,2024-09-28 10:00:00,0.002007,1


In [17]:
test_df = pd.DataFrame(
    columns = ["ds", "y", "unique_id"]   
)

test_df["ds"] = ohlcv["date"].iloc[-24:]
test_df["y"] = ohlcv["close_pct_change"].iloc[-24:]
test_df["unique_id"] = 1
test_df = test_df.reset_index(drop=True)

In [18]:
test_df

,ds,y,unique_id
0,2024-09-28 12:00:00,-0.003809,1
1,2024-09-28 13:00:00,-0.000201,1
2,2024-09-28 14:00:00,0.002878,1
3,2024-09-28 15:00:00,-0.001608,1
4,2024-09-28 16:00:00,-0.001623,1
5,2024-09-28 17:00:00,0.001878,1
6,2024-09-28 18:00:00,0.000981,1
7,2024-09-28 19:00:00,0.000463,1
8,2024-09-28 20:00:00,-0.001301,1
9,2024-09-28 21:00:00,0.000824,1


## MLFlow run

### ARCH

In [53]:
def make_arch_run(**kwargs):    
    with mlflow.start_run(run_name=f'ARCH_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "p": kwargs["p"], 
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "ARCH")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = StatsForecast(
            models=[ARCH(**params)],
            freq='h',
            n_jobs=-1
        )

        # Обучение моедли
        model.fit(train_df)

        # Прогнозирование для test датасета
        forecasts = model.forecast(48, level=[99, 95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlflavors.statsforecast.log_model(
            statsforecast_model=model,
            artifact_path="arch",
            serialization_format="pickle",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[99, 95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [54]:
make_arch_run(
    p=84
)

/usr/local/lib/python3.11/site-packages/statsforecast/core.py:510: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/statsforecast/core.py:528: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
2024/09/29 11:36:45 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpxuj_ug9e/model/model.pkl, flavor: statsforecast). Fall back to return ['statsforecast==1.7.7.1']. Set logging level to DEBUG to see the full traceback. 
2024/09/29 11:36:45 WARNING mlflow.models.model: Model logged without a signature an

### GARCH

In [55]:
def make_garch_run(**kwargs):    
    with mlflow.start_run(run_name=f'GARCH_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "p": kwargs["p"], 
            "q": kwargs["q"]
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "GARCH")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = StatsForecast(
            models=[GARCH(**params)],
            freq='h',
            n_jobs=-1
        )

        # Обучение моедли
        model.fit(train_df)

        # Прогнозирование для test датасета
        forecasts = model.forecast(48, level=[99, 95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlflavors.statsforecast.log_model(
            statsforecast_model=model,
            artifact_path="garch",
            serialization_format="pickle",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[99, 95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [56]:
make_garch_run(
    p=92, 
    q=24
)

/usr/local/lib/python3.11/site-packages/statsforecast/core.py:510: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/statsforecast/core.py:528: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
2024/09/29 11:36:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpu4kr6m15/model/model.pkl, flavor: statsforecast). Fall back to return ['statsforecast==1.7.7.1']. Set logging level to DEBUG to see the full traceback. 
2024/09/29 11:36:48 WARNING mlflow.models.model: Model logged without a signature an

### SVR

In [57]:
def make_svr_run(**kwargs):    
    with mlflow.start_run(run_name=f'SVR_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "kernel": kwargs["kernel"], 
            "degree": kwargs["degree"],
            "C": kwargs["C"]
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "SVR")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = MLForecast(
            models=[SVR(**params)],
            freq='h',  
            lags=list(range(1, 24, 1)),
        )
        
        # Обучение моедли
        model.fit(train_df, prediction_intervals=PredictionIntervals(n_windows=5, h=48))

        # Прогнозирование для test датасета
        #forecasts = model.predict(24, new_df=test_df)
        forecasts = model.predict(48, level=[99, 95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlforecast.flavor.log_model(
            model=model,
            artifact_path="scr",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[99, 95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [58]:
make_svr_run(
    kernel="poly", 
    degree=3,
    C=0.9702637495163653
)

2024/09/29 11:36:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/29 11:36:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run SVR_2024-09-29 11:36:49.588089 at: http://mlflow:5000/#/experiments/3/runs/b493c9200f5d46ddac0b50dddccc02b5.
2024/09/29 11:36:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/3.


### LGBMRegressor

In [59]:
def make_lightgbm_run(**kwargs):    
    with mlflow.start_run(run_name=f'LightGBM_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "n_estimators": kwargs["n_estimators"], 
            "boosting_type": kwargs["boosting_type"],
            "num_leaves": kwargs["num_leaves"],
            "max_depth": kwargs["max_depth"],
            "learning_rate": kwargs["learning_rate"],
            "verbose": -1
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "LightGBM")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = MLForecast(
            models=[lgb.LGBMRegressor(**params)],
            freq='h',  
            lags=list(range(1, 17, 1)),
        )

        # Обучение моедли
        model.fit(train_df, prediction_intervals=PredictionIntervals(n_windows=5, h=48))

        # Прогнозирование для test датасета
        #forecasts = model.predict(24, new_df=test_df)
        forecasts = model.predict(48, level=[99, 95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlforecast.flavor.log_model(
            model=model,
            artifact_path="lightgbm",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[99, 95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [60]:
make_lightgbm_run(
    n_estimators=382, 
    boosting_type="dart", 
    num_leaves=93, 
    max_depth=7, 
    learning_rate=0.04001572844964948
)

2024/09/29 11:37:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/29 11:37:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_2024-09-29 11:36:55.956921 at: http://mlflow:5000/#/experiments/3/runs/dd3dab93c54d4ebeafccf5867f4cd7ca.
2024/09/29 11:37:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/3.


### KNN

In [61]:
def make_knn_run(**kwargs):    
    with mlflow.start_run(run_name=f'KNN_{str(datetime.datetime.now())}') as run:
        # Лучшие параметры полученные в исследовании
        params = {
            "n_neighbors": kwargs["n_neighbors"], 
            "weights": kwargs["weights"],
            "leaf_size": kwargs["leaf_size"]
        }

        # Сохранение тегов
        mlflow.set_tag("model_name", "KNN")
        mlflow.set_tag("model_type", "regression")
        # Сохранение параметров
        mlflow.log_params(params)

        # Создание модели
        model = MLForecast(
            models=[KNeighborsRegressor(**params)],
            freq='h',  
            lags=list(range(1, 24, 1)),
        )

        # Обучение моедли
        model.fit(train_df, prediction_intervals=PredictionIntervals(n_windows=5, h=48))

        # Прогнозирование для test датасета
        #forecasts = model.predict(24, new_df=test_df)
        forecasts = model.predict(48, level=[99, 95, 90])
        forecasts["unique_id"] = 1

        # Рассчёт метрик
        rmse = losses.rmse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mse = losses.mse(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        mae = losses.mae(test_df["y"].values, forecasts.iloc[:, 2].values[:24])
        smape = losses.smape(test_df["y"].values, forecasts.iloc[:, 2].values[:24])

        # Сохранение метрик
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("SMAPE", smape)
        # Сохранение модели
        mlforecast.flavor.log_model(
            model=model,
            artifact_path="knn",
        )
        # Сохранение визуализации
        fig = plot_series(
            pd.concat([train_df, test_df]), 
            forecasts_df=forecasts,
            engine='matplotlib',
            level=[99, 95, 90],
        )
        fig.savefig('forecast.png', bbox_inches='tight')
        plt.close()
        mlflow.log_artifact("forecast.png", "forecast")

In [62]:
make_knn_run(
    n_neighbors=27, 
    weights='uniform', 
    leaf_size=34
)

2024/09/29 11:37:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/29 11:37:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run KNN_2024-09-29 11:37:02.271219 at: http://mlflow:5000/#/experiments/3/runs/83dc908603564c739d7a17076b7bdd08.
2024/09/29 11:37:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/3.


# P.s.

MLFlow load model не работает